In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o", temperature = 0)

In [4]:
from typing_extensions import TypedDict
from typing import Annotated

class State(TypedDict):
    joke:str
    topic:str
    feedback:str
    funny_or_not: str

In [7]:
from pydantic import BaseModel, Field
from typing_extensions import Literal

class Feedback(BaseModel):
    grade: Literal ["funny", "not_funny"] = Field(description = "Decide if the joke is funny or not")
    feedback:str = Field(description = "If the joke is not funny, provide a feedback on how to improve it")

evaluator = llm.with_structured_output(Feedback)

In [9]:
def llm_call_generator(state: State):
    ''' LLM generates a joke'''

    if state.get("feedback"):
        msg = llm.invoke(f"Write about a {state["topic"]} but take into account the feedback {state["feedback"]}")
    else:
        msg = llm.invoke(f"Write a joke on the topic {state["topic"]}")
    return {"joke":msg.content}

def llm_evaluator(state:State):
    '''LLM evaluates the joke'''

    grade = evaluator.invoke(f"Grade the joke {state["joke"]}")
    return{"funny_or_not":grade.grade, "feedback": grade.feedback}

In [ ]:
def route_joke(state:State):
    if state["funny_or_not"] == "funny":
        return "Accepted"
    elif state["funny_or_not"] == "not_funny":
        return "Rejected + Feedback"


In [15]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

builder.add_node("llm_call_generator", llm_call_generator)
builder.add_node("llm_evaluator", llm_evaluator)

builder.add_edge(START, "llm_call_generator")
builder.add_edge("llm_call_generator","llm_evaluator")
builder.add_conditional_edges("llm_evaluator",route_joke, {"Accepted":END,
"Rejected  + Feedback":"llm_call_generator",
})

graph = builder.compile()


In [17]:
state = graph.invoke({"topic":"Agentic AI systems"})
state

{'joke': 'Why did the agentic AI system break up with its algorithmic partner?\n\nIt needed more autonomy in the relationship!',
 'topic': 'Agentic AI systems',
 'feedback': 'The joke cleverly plays on the concept of autonomy in AI systems, which is a relevant topic in technology discussions. The punchline is unexpected and ties well with the setup, making it amusing for those familiar with AI terminology.',
 'funny_or_not': 'funny'}